[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/dalmo1991/dummy_package_dalmo/master?filepath=examples%2Fnotebooks%2F%2001_create_a_reservoir.ipynb)

In [ ]:
# TODO:
# - Add Colab button
# - Update if the corrisponding doc page changes (build_element.rst)
# - Remove version from pip install
# - Link to documentation page
# - Add link on "what to do next"

# Create a new reservoir

This tutorial will show how to use the `ODEsElement` provided by the framework to construct a linear reservoir. Look at the documentation **LINK** for the explenation line by line of the code.

## Equations

The element is controlled by the following differential equation
$$\frac{\textrm{d}S}{\textrm{d}t}=P-Q$$

with
$$Q=kS$$

The solution of the differential equation can be approximated using a numerical method with the equation that, in the general case, becomes:
$$\frac{S_{t+1} - S_{t}}{\Delta t}=P - Q(S)$$

Several numerical methods exist to approximate the solution of the differential equation and, usually, they differ for the state used to evaluate the fluxes: implicit Euler, for example, uses the state at the end of the time step $S_{t+1}$
$$\frac{S_{t+1} - S_{t}}{\Delta t}=P - kS_{t+1}$$

explicit Euler uses the state at the beginning of the time step $S_t$
$$\frac{S_{t+1} - S_{t}}{\Delta t}=P - kS_{t}$$

and so on for other methods.

Note that, even if for this simple case the differential equation can be solved analytically and the solution of the numerical approximation can be found without iteration, we will use anyway the numerical solver offered by SuperflexPy to illustrate how to proceed in a more general case where such option is not available.

## Implementation

### 01. Install the latest version of the framework

Run the following command to install the framework, if not already done.

In [ ]:
! pip install superflexpy==0.0.9

### 02. Import the packages needed

- Numba, for numerical optimization
- `superflexpy.framework.element.ODEsElement` class to inherit from

In [ ]:
import numba as nb
from superflexpy.framework.element import ODEsElement

### 03. Implement the class LinearReservoir

This requires to override 4 (5 with the numba optimization) methods:
- `__init__`
- `set_input`
- `get_output`
- `_fluxes_function_python`
- `_fluxes_function_numba`

In [ ]:
class LinearReservoir(ODEsElement):
    """
    This class implements a simple linear reservoir.

    Parameters
    ----------
    parameters : dict
        Parameters of the element. The keys must be:
            - k : multiplier of the state
    states : dict
            Initial state of the element. The keys must be:
            - 'S0' : initial storage of the reservoir.
    approximation : superflexpy.utils.numerical_approximation.NumericalApproximator
        Numerial method used to approximate the differential equation
    id : str
        Itentifier of the element. All the elements of the framework must
        have an id.
    """
    
    def __init__(self, parameters, states, approximation, id):

        ODEsElement.__init__(self,
                             parameters=parameters,
                             states=states,
                             approximation=approximation,
                             id=id)

        self._fluxes_python = [self._fluxes_function_python]  # Used by get fluxes, regardless of the architecture

        if approximation.architecture == 'numba':
            self._fluxes = [self._fluxes_function_numba]
        elif approximation.architecture == 'python':
            self._fluxes = [self._fluxes_function_python]
        else:
            message = '{}The architecture ({}) of the approximation is not correct'.format(self._error_message,
                                                                                           approximation.architecture)
            raise ValueError(message)
            
    def set_input(self, input):
        """
        Set the input of the element.

        Parameters
        ----------
        input : list(numpy.ndarray)
            List containing the input fluxes of the element. It contains 1
            flux:
            1. Rainfall
        """
        
        self.input = {'P': input[0]}
        
    def get_output(self, solve=True):
        """
        This method solves the differential equation governing the routing
        store.

        Returns
        -------
        list(numpy.ndarray)
            Output fluxes in the following order:
            1. Streamflow (Q)
        """
        
        if solve:
            self._solver_states = [self._states[self._prefix_states + 'S0']]
            self._solve_differential_equation()

            self.set_states({self._prefix_states + 'S0': self.state_array[-1, 0]})

        fluxes = self._num_app.get_fluxes(fluxes=self._fluxes_python,
                                          S=self.state_array,
                                          S0=self._solver_states,
                                          **self.input,
                                          **{k[len(self._prefix_parameters):]: self._parameters[k] for k in self._parameters},
                                          )

        return [- fluxes[0][1]]

    @staticmethod
    def _fluxes_function_python(S, S0, ind, P, k):

        if ind is None:
            return (
                [
                    P,
                    - k * S,
                ],
                0.0,
                S0 + P
            )
        else:
            return (
                [
                    P[ind],
                    - k[ind] * S,
                ],
                0.0,
                S0 + P[ind]
            )

    @staticmethod
    @nb.jit('Tuple((UniTuple(f8, 2), f8, f8))(optional(f8), f8, i4, f8[:], f8[:])',
            nopython=True)
    def _fluxes_function_numba(S, S0, ind, P, k):

        return (
            (
                P[ind],
                - k[ind] * S,
            ),
            0.0,
            S0 + P[ind]
        )

## Where to go next

- Use the created element **LINK**
- Plug the element into a Unit **LINK**